# Evaluación Modular 7:
# Sistema Inteligente de Scoring Crediticio con Redes Neuronales Profundas

## Objetivo
Diseñar, entrenar y evaluar un modelo de red neuronal profunda para predecir la probabilidad de impago de clientes bancarios, utilizando un conjunto de datos realista. El modelo debe ser explicable, eficiente y presentar resultados interpretables para su uso en contextos financieros.

**Datasets utilizados:**  
`German Credit Data`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y análisis exploratorio de datos:**
    - Se carga el dataset **German Credit Data** y se nombran las columnas según la descripción oficial.
    - Se realiza un análisis inicial de las variables:
        - Descripción estadística de variables numéricas y categóricas.
        - Distribución de la variable objetivo (`credit_risk`) y balance de clases.
        - Visualización de conteos para la variable target.

2. **Preprocesamiento de datos y manejo de desbalanceo:**
    - Mapeo de variables ordinales según los diccionarios oficiales.
    - Codificación **One-Hot** para variables nominales.
    - Escalado de variables numéricas con **StandardScaler**.
    - División en conjuntos **train/test** estratificados.
    - Cálculo de **pesos de clase** para balancear la función de pérdida durante el entrenamiento de los modelos.

3. **Construcción de modelos de predicción:**
    - Se construye un **Simple DNN** con capas densas, activación ReLU, regularización L2 y Dropout.
    - Se construye un **ResNet para datos tabulares** con bloques residuales, activación ReLU y salida sigmoide.
    - Compilación de los modelos con **binary cross-entropy** y optimizador Adam.

4. **Entrenamiento de modelos:**
    - Entrenamiento con **early stopping** y reducción adaptativa del learning rate.
    - Uso de los **pesos de clase** para manejar el desbalanceo.
    - Validación durante el entrenamiento en el conjunto de test.

5. **Evaluación de desempeño:**
    - Cálculo de métricas de clasificación:
        - **Accuracy, Precision, Recall, F1-score y ROC-AUC**.
    - Comparación de los modelos en una tabla resumida.

6. **Explicabilidad con SHAP:**
    - Uso de **SHAP KernelExplainer** para analizar la contribución de cada variable a las predicciones.
    - Selección de un subconjunto de muestras de test para los cálculos de SHAP.
    - Visualización de **summary plots** para cada modelo, mostrando la importancia relativa de variables numéricas y nominales (one-hot).

7. **Interpretación y análisis financiero:**
    - Evaluación de la importancia relativa de **falsos positivos y falsos negativos** según el contexto financiero.
    - Discusión sobre qué variables influyen más en el riesgo de crédito.

---

# 2. Configuración del entorno

--- 

In [ ]:
# Librerías
import os, io, contextlib, random, tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import shap

# Reproducibilidad y disminución de logs
os.environ['PYTHONHASHSEED'] = '42'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
tf.get_logger().setLevel('ERROR')
tqdm.tqdm = lambda *args, **kwargs: iter([])

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga, preprocesamiento de datos y aplicación de reducción de dimensionalidad de los datos.

- **`cargar_y_analizar_datos()`** 
Carga y analiza el dataset German Credit mostrando estadísticas y distribución de clases.

- **`preprocesar_datos()`** 
Preprocesa datos (ordinal, nominal, escalado, train/test) y calcula pesos de clase.

---

### **Decisiones de diseño**

##### **Definición de nombres de columnas:**

- El archivo `german.data` no incluye encabezados, por lo que al cargarlo con pandas sería imposible referirse a las variables por nombre. Definir los nombres de columnas manualmente garantiza que cada atributo tenga una etiqueta clara y permite manipular los datos de forma más legible y menos propensa a errores.

##### **División en variables numéricas, ordinales y nominales:**

- En preprocesamiento, no basta con separar solo en numéricas y categóricas, porque las categóricas pueden tener orden lógico (ordinales) o no tenerlo (nominales). Las ordinales tienen categorías con jerarquía o progresión (ejemplo: nivel de ahorro o antigüedad laboral). Aquí es válido mapearlas a números enteros respetando su orden. Por otro lado, las nominales son solo etiquetas sin orden (ejemplo: propósito del crédito), y deben procesarse con One-Hot Encoding para evitar imponer un orden artificial.

- La clasificación de cada variable como ordinal o nominal se hizo usando la documentación oficial del dataset, donde se especifica el significado y el orden implícito de cada atributo.

##### **Cambio de valores de target de 1 y 2 a 0 y 1**

- La conversión de las etiquetas de credit_risk de 1=good y 2=bad a 0 y 1 es una práctica común en clasificación binaria. Muchos algoritmos y métricas esperan que la clase negativa sea 0 y la positiva sea 1, lo que evita errores en bibliotecas como scikit-learn y simplifica el manejo de métricas como accuracy, precision o ROC-AUC.

##### **Balanceo de clases**

- El dataset original tiene un desbalance de ~70% clase "good" y ~30% "bad". Esto puede sesgar el modelo para predecir mayoritariamente la clase mayoritaria.

- Se optó por class weights en lugar de técnicas de sobremuestreo como SMOTE porque:
    - Mantiene intacta la distribución real de datos.
    - Evita generar datos sintéticos que podrían no representar fielmente casos de crédito real.
    - Es más simple y rápido, y funciona bien con la mayoría de algoritmos basados en gradiente o redes neuronales.

- Los class weights le dicen al modelo que penalice más los errores en la clase minoritaria, equilibrando el aprendizaje sin alterar los datos originales.

---

In [ ]:
def cargar_y_analizar_datos(filepath='german.data'):
    """
    Carga el dataset German Credit Data, analiza sus variables y distribuciones, 
    y muestra estadísticas básicas y gráficos de la clase objetivo.

    Pasos:
    1. Definir nombres de columnas y cargar el CSV.
    2. Mostrar primeras filas, descripción general y tipo de datos.
    3. Visualizar la distribución de la variable objetivo 'credit_risk'.

    Args:
        filepath (str): Ruta al archivo de datos. Por defecto 'german.data'.

    Returns:
        pd.DataFrame: DataFrame completo con los datos cargados.
    """
    column_names = [
        "checking_status", "duration", "credit_history", "purpose",
        "credit_amount", "savings", "employment", "installment_rate",
        "personal_status", "other_parties", "residence_since", "property",
        "age", "other_installments", "housing", "existing_credits",
        "job", "num_dependents", "telephone", "foreign_worker", "credit_risk"
    ]
    df = pd.read_csv(filepath, sep=' ', names=column_names)

    print("Primeras filas del dataset:")
    display(df.head())
    print("\nDescripción general (numéricas y categóricas):")
    display(df.describe(include='all'))
    print("\nInformación general:")
    display(df.info())

    print("\nDistribución de clases (credit_risk):")
    print(df['credit_risk'].value_counts())
    sns.countplot(x='credit_risk', data=df)
    plt.title("Distribución de clases: good vs bad")
    plt.show()

    return df

def preprocesar_datos(df):
    """
    Preprocesa los datos del German Credit Data, incluyendo:
    - Mapeo de variables ordinales.
    - One-Hot Encoding de variables nominales.
    - Escalado de variables numéricas.
    - División estratificada en train/test.
    - Cálculo de pesos de clase para balanceo.

    Pasos:
    1. Mapear variables ordinales a valores enteros.
    2. Definir variables nominales para One-Hot Encoding.
    3. Construir pipelines para transformación numérica y categórica.
    4. Ajustar y transformar train/test.
    5. Calcular pesos de clase.

    Args:
        df (pd.DataFrame): DataFrame original con los datos cargados.

    Returns:
        X_train_prep (np.ndarray): Features de entrenamiento preprocesadas.
        X_test_prep (np.ndarray): Features de test preprocesadas.
        y_train (np.ndarray): Etiquetas de entrenamiento.
        y_test (np.ndarray): Etiquetas de test.
        preprocessor (ColumnTransformer): Pipeline de preprocesamiento ajustado.
        class_weight_dict (dict): Diccionario con pesos de clase para entrenamiento.
    """
    target = 'credit_risk'

    ordinal_vars_maps = {
        'checking_status': {'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3},
        'savings': {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4},
        'employment': {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4},
        'personal_status': {'A91': 0, 'A92': 1, 'A93': 2, 'A94': 3, 'A95': 4},
        'property': {'A121': 0, 'A122': 1, 'A123': 2, 'A124': 3},
        'other_installments': {'A141': 0, 'A142': 1, 'A143': 2},
        'housing': {'A151': 0, 'A152': 1, 'A153': 2},
        'job': {'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3},
        'telephone': {'A191': 0, 'A192': 1},
        'foreign_worker': {'A201': 1, 'A202': 0}
    }

    for col in ordinal_vars_maps.keys():
        df[col] = df[col].map(ordinal_vars_maps[col]).astype(int)

    nominal_vars = ['credit_history', 'purpose', 'other_parties']
    num_vars = [col for col in df.columns if col not in nominal_vars + [target]]

    print("Variables numéricas (incluidas ordinales):", num_vars)
    print("Variables nominales para One-Hot:", nominal_vars)

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, num_vars),
        ('cat', categorical_transformer, nominal_vars)
    ])

    X = df.drop(columns=[target])
    y = df[target].map({1: 0, 2: 1})  # good=0, bad=1

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42)

    X_train_prep = preprocessor.fit_transform(X_train)
    X_test_prep = preprocessor.transform(X_test)

    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(enumerate(class_weights))
    print("Pesos de clase calculados:", class_weight_dict)

    return X_train_prep, X_test_prep, y_train.values, y_test.values, preprocessor, class_weight_dict

**Bloque 2:** Creación, entrenamiento y evaluación de modelos.

- **`construir_dnn_simple()`** 
Crea y compila una red neuronal densa simple para clasificación binaria, con capas ocultas densas, regularización L2 y capas de dropout para prevenir sobreajuste.

- **`bloque_residual()`** 
Implementa un bloque residual totalmente conectado. Permite el flujo directo de información mediante conexiones de salto (skip connections), ayudando a entrenar redes más profundas y estables.

- **`construir_resnet_tabular()`** 
Crea y compila una red neuronal tipo ResNet adaptada a datos tabulares, utilizando bloques residuales para mejorar la capacidad de generalización y mitigar el problema del desvanecimiento del gradiente.

- **`entrenar_modelo()`** 
Entrena un modelo usando Early Stopping (para detener el entrenamiento cuando no hay mejora) y reducción adaptativa de la tasa de aprendizaje (ReduceLROnPlateau), ajustando el entrenamiento al desbalanceo de clases mediante class_weight.

- **`evaluar_modelo()`** 
Evalúa el rendimiento del modelo sobre datos de prueba, calculando métricas clave de clasificación binaria como accuracy, precision, recall, f1-score y ROC AUC, además de devolver las probabilidades predichas.

---

### **Decisiones de diseño**

##### **Diseño de la DNN simple:**

- Se optó por una red neuronal densa (DNN) con dos capas ocultas de 128 y 64 neuronas respectivamente, usando activación ReLU y regularización L2 combinada con dropout. Este diseño es suficiente para capturar relaciones no lineales en datos tabulares sin caer en una complejidad excesiva que incremente el riesgo de sobreajuste. La regularización L2 penaliza pesos grandes, y el dropout fuerza a la red a no depender en exceso de neuronas específicas, mejorando la capacidad de generalización.

##### **Propósito del bloque residual:**

- El bloque residual implementa skip connections, permitiendo que la señal original se sume a la salida de las capas intermedias. Esto facilita el entrenamiento de redes más profundas al mitigar el problema del desvanecimiento del gradiente y permite que la red aprenda funciones de identidad cuando es necesario, lo que aumenta su estabilidad y capacidad de adaptación.

##### **Uso de ResNet tabular:**

- La variante ResNet adaptada a datos tabulares permite aprovechar la estabilidad y la eficiencia en el entrenamiento que ofrecen los bloques residuales, especialmente en modelos con más capas. Esta arquitectura moderna suele superar a DNN simples en tareas complejas, ya que evita la degradación del rendimiento al aumentar la profundidad de la red.

##### **Justificación para Early Stopping y ReduceLROnPlateau:**

- Early Stopping: Detiene el entrenamiento cuando el rendimiento en el conjunto de validación deja de mejorar, evitando así el sobreajuste y reduciendo el tiempo de entrenamiento innecesario.

- ReduceLROnPlateau: Disminuye la tasa de aprendizaje cuando el progreso se estanca, permitiendo que el optimizador realice ajustes más finos y aumente la probabilidad de converger a un mínimo óptimo.

##### **Justificación de las métricas utilizadas:**

- Accuracy: Proporción total de predicciones correctas; útil como referencia general.
- Precision: Proporción de predicciones positivas correctas; relevante cuando el costo de un falso positivo es alto.
- Recall: Proporción de casos positivos correctamente detectados; esencial cuando es crítico no omitir positivos reales.
- F1-score: Media armónica de precisión y recall; balancea ambas métricas y es clave en escenarios con clases desbalanceadas.
- ROC AUC: Evalúa la capacidad de discriminación del modelo entre clases para todos los umbrales posibles, proporcionando una medida robusta de rendimiento global.

---

In [ ]:
def construir_dnn_simple(forma_entrada):
    """
    Construye y compila un modelo DNN (red neuronal densa) simple para clasificación binaria.
    
    Parámetros:
        forma_entrada (tuple): Forma de los datos de entrada (n_features,).

    Retorna:
        tensorflow.keras.Model: Modelo compilado listo para entrenamiento.
    """
    modelo = models.Sequential([
        layers.Input(shape=forma_entrada),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo


def bloque_residual(x, unidades):
    """
    Implementa un bloque residual con dos capas densas y conexión de atajo.
    
    Parámetros:
        x (tensor): Entrada al bloque.
        unidades (int): Número de neuronas en las capas densas.

    Retorna:
        tensor: Salida del bloque residual.
    """
    atajo = x
    x = layers.Dense(unidades, activation='relu')(x)
    x = layers.Dense(unidades)(x)
    x = layers.Add()([x, atajo])
    x = layers.Activation('relu')(x)
    return x


def construir_resnet_tabular(forma_entrada):
    """
    Construye y compila un modelo tipo ResNet adaptado para datos tabulares.
    
    Parámetros:
        forma_entrada (tuple): Forma de los datos de entrada (n_features,).

    Retorna:
        tensorflow.keras.Model: Modelo compilado listo para entrenamiento.
    """
    entradas = layers.Input(shape=forma_entrada)
    x = layers.Dense(64, activation='relu')(entradas)
    x = bloque_residual(x, 64)
    x = bloque_residual(x, 64)
    x = layers.Dense(1, activation='sigmoid')(x)
    modelo = models.Model(inputs=entradas, outputs=x)
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo


def entrenar_modelo(modelo, X_entrenamiento, y_entrenamiento, X_validacion, y_validacion, peso_clase):
    """
    Entrena un modelo Keras con early stopping y reducción adaptativa de tasa de aprendizaje.
    
    Parámetros:
        modelo (tensorflow.keras.Model): Modelo compilado a entrenar.
        X_entrenamiento (array): Datos de entrenamiento.
        y_entrenamiento (array): Etiquetas de entrenamiento.
        X_validacion (array): Datos de validación.
        y_validacion (array): Etiquetas de validación.
        peso_clase (dict): Pesos para balancear clases.

    Retorna:
        tensorflow.keras.callbacks.History: Historial de entrenamiento.
    """
    early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-6)
    historial = modelo.fit(
        X_entrenamiento, y_entrenamiento,
        epochs=50,
        batch_size=32,
        validation_data=(X_validacion, y_validacion),
        class_weight=peso_clase,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    return historial

def evaluar_modelo(modelo, X_prueba, y_prueba):
    """
    Evalúa un modelo de clasificación binaria y calcula métricas de rendimiento.
    
    Parámetros:
        modelo (tensorflow.keras.Model): Modelo entrenado.
        X_prueba (array): Datos de prueba.
        y_prueba (array): Etiquetas reales.

    Retorna:
        tuple: 
            - dict con métricas (accuracy, precision, recall, f1_score, roc_auc)
            - array con probabilidades predichas.
    """
    y_pred_prob = modelo.predict(X_prueba).flatten()
    y_pred = (y_pred_prob >= 0.5).astype(int)
    metricas = {
        'accuracy': accuracy_score(y_prueba, y_pred),
        'precision': precision_score(y_prueba, y_pred),
        'recall': recall_score(y_prueba, y_pred),
        'f1_score': f1_score(y_prueba, y_pred),
        'roc_auc': roc_auc_score(y_prueba, y_pred_prob)
    }
    return metricas, y_pred_prob


**Bloque 3:** Visualización de resultados e importancia de variables.

- **`predecir_shap_silenciado()`** 
Función auxiliar que permite obtener predicciones del modelo sin mostrar salidas por consola, necesaria para evitar ruido durante el cálculo de valores SHAP.

- **`graficar_shap()`** 
Genera gráficos de resumen SHAP para interpretar la importancia relativa de las características en distintos modelos. Facilita la comparación visual de interpretabilidad.

- **`mostrar_tabla_comparativa()`** 
Presenta en formato tabular las métricas de rendimiento de un modelo DNN simple frente a un modelo ResNet tabular, permitiendo una comparación rápida.

---

### **Decisiones de diseño**

##### **Justificación de `predecir_shap_silenciado()`:**

- Durante el cálculo de valores SHAP, la función interna de predicción del modelo (model.predict) imprime múltiples logs en consola, especialmente cuando se trabaja con Keras/TensorFlow. Estos mensajes saturaban la salida y mezclaban información irrelevante con los gráficos SHAP, dificultando la interpretación visual. Por eso, se creó `predecir_shap_silenciado()`, que redirige la salida estándar a un buffer temporal, evitando que los logs interfieran con los resultados y manteniendo la consola limpia.

##### **Uso de SHAP para mostrar la importancia de las variables:**

- SHAP (SHapley Additive exPlanations) es una técnica basada en teoría de juegos que asigna una contribución a cada característica según su impacto en la predicción del modelo.

- Se utiliza porque:
    - Interpretabilidad global y local → Permite entender tanto el peso de cada variable en el modelo completo como en predicciones individuales.
    - Consistencia matemática → Los valores SHAP garantizan que si una característica tiene mayor contribución, su valor asignado será siempre mayor que el de otra con menor influencia.
    - Detección de relaciones no lineales → A diferencia de la simple importancia de características en árboles, SHAP captura interacciones complejas.
    - Transparencia y confianza → Facilita explicar las decisiones del modelo a usuarios no técnicos, mejorando la trazabilidad en contextos críticos (salud, finanzas, etc.).

---

In [ ]:
def predecir_shap_silenciado(modelo, X):
    """
    Realiza predicciones con un modelo silenciando la salida estándar 
    para evitar impresiones innecesarias durante el cálculo con SHAP.

    Parámetros:
        modelo: Modelo entrenado de Keras.
        X (ndarray): Datos de entrada.

    Retorna:
        ndarray: Predicciones aplanadas del modelo.
    """
    with contextlib.redirect_stdout(io.StringIO()):
        predicciones = modelo.predict(X)
    return predicciones.flatten()


def graficar_shap(lista_modelos, X_muestra, nombres_caracteristicas, titulos):
    """
    Genera gráficos de resumen SHAP para interpretar la importancia de 
    las características en varios modelos.

    Parámetros:
        lista_modelos (list): Lista de modelos entrenados.
        X_muestra (ndarray): Subconjunto de datos para explicación.
        nombres_caracteristicas (list): Nombres de las variables.
        titulos (list): Títulos para cada gráfico.
    """
    fondo = X_muestra[np.random.choice(X_muestra.shape[0], 
                                       min(50, X_muestra.shape[0]), 
                                       replace=False)]
    for i, modelo in enumerate(lista_modelos):
        print(f"\nGráfico SHAP para: {titulos[i]}")
        explicador = shap.KernelExplainer(lambda x: predecir_shap_silenciado(modelo, x), fondo)
        valores_shap = explicador.shap_values(X_muestra, nsamples=100, progress=False)
        shap.summary_plot(valores_shap, X_muestra, feature_names=nombres_caracteristicas, show=True)


def mostrar_tabla_comparativa(metricas_simple, metricas_resnet):
    """
    Muestra una tabla comparativa de métricas entre un modelo DNN simple y un ResNet tabular.

    Parámetros:
        metricas_simple (dict): Métricas del modelo DNN simple.
        metricas_resnet (dict): Métricas del modelo ResNet tabular.
    """
    df_metricas = pd.DataFrame([metricas_simple, metricas_resnet], 
                               index=['DNN Simple', 'ResNet Tabular'])
    display(df_metricas)


**Bloque 4:** Función de ejecución.

- **`main()`** 
Función orquestadora que ejecuta todo el pipeline: carga y análisis de datos, preprocesamiento, construcción y entrenamiento de modelos, evaluación y comparación de métricas, análisis de explicabilidad con SHAP y recordatorio para análisis financiero.

---

In [ ]:
def main():
    """
    Orquesta todo el flujo de trabajo del proyecto:
    1. Carga y análisis inicial de los datos.
    2. Preprocesamiento (división, escalado, codificación y pesos de clases).
    3. Construcción de modelos (DNN simple y ResNet tabular).
    4. Entrenamiento de modelos.
    5. Evaluación y comparación de métricas.
    6. Explicabilidad mediante SHAP.
    7. Mensaje de recordatorio para análisis de impacto financiero.
    """
    print("=== Cargando y analizando datos ===")
    datos = cargar_y_analizar_datos()

    print("=== Preprocesando datos ===")
    X_entrenamiento, X_prueba, y_entrenamiento, y_prueba, preprocesador, pesos_clase = preprocesar_datos(datos)

    forma_entrada = X_entrenamiento.shape[1]
    print(f"Dimensión de entrada para modelos: {forma_entrada}")

    print("=== Construyendo modelos ===")
    modelo_simple = construir_dnn_simple(forma_entrada)
    modelo_resnet = construir_resnet_tabular(forma_entrada)

    print("=== Entrenando modelo DNN simple ===")
    entrenar_modelo(modelo_simple, X_entrenamiento, y_entrenamiento, X_prueba, y_prueba, pesos_clase)

    print("=== Entrenando modelo ResNet tabular ===")
    entrenar_modelo(modelo_resnet, X_entrenamiento, y_entrenamiento, X_prueba, y_prueba, pesos_clase)

    print("=== Evaluando modelos ===")
    metricas_simple, y_prob_simple = evaluar_modelo(modelo_simple, X_prueba, y_prueba)
    metricas_resnet, y_prob_resnet = evaluar_modelo(modelo_resnet, X_prueba, y_prueba)

    mostrar_tabla_comparativa(metricas_simple, metricas_resnet)

    print("=== Explicabilidad con SHAP ===")
    # Subconjunto para SHAP (ejemplo: primeras 100 muestras)
    X_shap_muestra = X_prueba[:100]

    # Nombres de características (numéricas + OneHot nominales)
    nombres_caracteristicas = list(preprocesador.transformers_[0][2])  # numéricas
    nombres_onehot = preprocesador.named_transformers_['cat']['onehot'].get_feature_names_out(
        preprocesador.transformers_[1][2]
    )
    nombres_caracteristicas.extend(nombres_onehot)

    graficar_shap(
        [modelo_simple, modelo_resnet],
        X_shap_muestra,
        nombres_caracteristicas=nombres_caracteristicas,
        titulos=['DNN Simple', 'ResNet Tabular']
    )

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## Evaluación de las métricas obtenidas

Las métricas de rendimiento, como la precisión y el recall, fueron bajas para ambos modelos, indicando que el dataset representa un desafío. El dataset german.data es conocido por ser pequeño (solo 1000 instancias), desbalanceado y de baja dimensionalidad.
- Tamaño del Dataset: Con solo 1000 instancias, el modelo tiene un número limitado de ejemplos para aprender los patrones subyacentes. Esto aumenta la probabilidad de sobreajuste y dificulta la generalización a datos no vistos.
- Desbalance de Clases: El desbalance de clases (700 casos de "buen crédito" frente a 300 de "mal crédito") complica el entrenamiento. Si bien se usaron pesos de clase para mitigar este problema, el modelo aún puede tener dificultades para aprender las características de la clase minoritaria ("mal crédito").
- Naturaleza de los Datos: El dataset consta de 20 variables, muchas de ellas categóricas. La transformación one-hot encoding crea nuevas variables, pero los patrones no lineales y las interacciones entre ellas son difíciles de capturar para cualquier modelo, especialmente con un conjunto de datos pequeño.

La DNN simple obtuvo un rendimiento ligeramente superior, especialmente en la métrica de recall, que es la más importante en este contexto.
- Complejidad del Modelo: Una arquitectura más simple, como la DNN simple, puede ser más adecuada para un dataset pequeño y ruidoso. Una ResNet tabular, al ser una arquitectura más compleja con conexiones residuales, podría estar sobreajustándose a los datos de entrenamiento.
- Sobreadaptación (Overfitting): La ResNet tabular es ideal para problemas más complejos y datasets más grandes donde las conexiones residuales pueden ayudar a mitigar el problema del gradiente evanescente y permitir que la red se profundice sin degradar el rendimiento. En un dataset pequeño, su complejidad podría haber capturado el ruido en lugar de las señales, lo que se traduce en una menor capacidad de generalización. La DNN simple tiene menos parámetros, lo que reduce el riesgo de sobreajuste y puede resultar en un modelo más robusto para este dataset en particular.

---

## Análisis de resultados en el contexto del impacto de errores tipo I y II

En el ámbito financiero, los errores de clasificación tienen un impacto económico directo.
- Error Tipo I (Falso Positivo): Ocurre cuando se deniega un crédito a un cliente que en realidad es de bajo riesgo. Esto se traduce en una pérdida de ingresos potenciales para el banco. En nuestras métricas, este error se relaciona con la precisión. Un valor bajo de precisión indica una alta tasa de Falsos Positivos.
- Error Tipo II (Falso Negativo): Ocurre cuando se aprueba un crédito a un cliente que en realidad es de alto riesgo y terminará por no pagarlo. Esto se traduce en una pérdida de capital directa para el banco. Este error se relaciona con el recall. Un valor bajo de recall indica una alta tasa de Falsos Negativos.

En este contexto, podemos evaluar los dos modelos probados en este trabajo:
- DNN simple:
  - Recall (0.70): La DNN simple es mejor detectando a los clientes que realmente son malos créditos (clase 1). Captura el 70% de los casos de "mal crédito". Esto significa que su tasa de Falsos Negativos (Errores Tipo II) es menor que la de la ResNet. Desde una perspectiva financiera, esto es muy valioso, ya que reduce la pérdida de capital al evitar otorgar préstamos a una mayor proporción de clientes que no pagarían.
  - Precision (0.55): Sin embargo, al ser más "sensible", también clasifica incorrectamente a más clientes como malos créditos (Falsos Positivos). Esto se refleja en su precisión, lo que indica que pierde más ingresos potenciales al rechazar a clientes que sí podrían pagar.
- ResNet Tabular:
  - Recall (0.62): La ResNet tabular tiene un recall más bajo, lo que significa que su tasa de Falsos Negativos (Errores Tipo II) es más alta. Detecta menos de los malos créditos reales, lo que podría llevar a mayores pérdidas de capital por impago.
  - Precision (0.54): Su precisión es ligeramente inferior, pero no de forma significativa. Esto sugiere un comportamiento similar en términos de Falsos Positivos.

En el contexto financiero, donde los errores tipo II (otorgar un mal crédito) son generalmente mucho más costosos que los errores tipo I (denegar un buen crédito), el modelo Simple DNN sería la mejor opción. Su mayor recall (70% vs. 62%) indica que es más efectivo a la hora de identificar a los clientes de alto riesgo, lo que se traduce directamente en una menor exposición a pérdidas por impagos.

Aunque la DNN simple tiene una precisión ligeramente menor, el costo de un impago suele ser tan elevado que la prioridad es minimizar el recall. Si bien se perderán algunos clientes potenciales (Falsos Positivos), se evitará una mayor cantidad de préstamos incobrables.

## Análisis de la importancia de variables visualizadas con SHAP

Los gráficos SHAP (SHapley Additive exPlanations) que proporcionaste visualizan la importancia e impacto de cada característica en la predicción de los dos modelos de redes neuronales (una DNN simple y una ResNet tabular) para el riesgo de crédito. Un punto en el gráfico representa una instancia individual del conjunto de datos.

- El eje X muestra el valor SHAP, que es el impacto de la característica en el "output" (predicción) del modelo.
  - Valores positivos impulsan la predicción a ser un mal crédito (el target y=1).
  - Valores negativos impulsan la predicción a ser un buen crédito (el target y=0).
- El eje Y lista las características del conjunto de datos, ordenadas por su importancia media.
- El color de los puntos indica el valor real de la característica para esa instancia.
  - El rojo (High) indica un valor alto de la característica.
  - El azul (Low) indica un valor bajo de la característica.

### Comparación de variables entre Modelos (DNN vs. ResNet)

Ambos gráficos identifican las mismas características principales como las más influyentes, lo cual es un buen indicio de que ambos modelos están aprendiendo patrones similares en los datos. No obstante, hay sutiles diferencias que vale la pena notar.

En ambos modelos, las características más importantes para predecir el riesgo de crédito son:
- checking_status: Es la característica más influyente con diferencia.
  - Valores altos (rojo) (indicando una cuenta corriente con poco o ningún saldo) tienen un impacto positivo en el valor SHAP, lo que significa que aumentan la probabilidad de ser un mal crédito.
  - Valores bajos (azul) (indicando un buen saldo) tienen un impacto negativo, disminuyendo la probabilidad de ser un mal crédito. Este es un resultado intuitivo y esperado en el análisis de riesgo crediticio.
- duration (duración del crédito):
  - Valores altos (rojo) (créditos a largo plazo) tienen un impacto positivo y están asociados a un mayor riesgo.
  - Valores bajos (azul) (créditos a corto plazo) tienen un impacto negativo, sugiriendo menor riesgo. Esto también es coherente con la práctica bancaria, donde plazos más largos implican mayor incertidumbre.
- savings (ahorros):
  - Valores altos (rojo) (ahorros considerables) tienen un impacto negativo en el valor SHAP, reduciendo el riesgo de ser un mal crédito.
  - Valores bajos (azul) (pocos o ningún ahorro) tienen un impacto positivo, aumentando el riesgo.

---

## Reflexiones

### Ética, Sesgos Posibles y Decisiones no Explicadas

El modelo de riesgo crediticio, si no se gestiona con cuidado, puede perpetuar y amplificar sesgos existentes en los datos.

- Sesgos en los datos: El conjunto de datos german.data ya podría contener sesgos históricos. Por ejemplo, si los bancos históricamente han denegado más créditos a ciertos grupos demográficos, el modelo podría aprender esta correlación y continuar denegándolos, aunque no haya una base real en la solvencia. Las variables como age, job, personal_status y residence_since son especialmente sensibles al sesgo. Por ejemplo, si el modelo usa age para inferir que los jóvenes son de mayor riesgo, podría estar tomando una decisión sesgada.

- Decisiones no explicadas (Black Box): Las redes neuronales profundas, como las que usamos, son a menudo "cajas negras" (black box). Esto significa que no es fácil entender por qué el modelo tomó una decisión específica para un individuo. Sin herramientas de explicabilidad, un oficial de crédito no podría justificar la denegación de un préstamo más allá de decir: "El modelo lo predijo así". Esto es éticamente problemático y puede llevar a la discriminación sin intención. La ausencia de transparencia dificulta la auditoría y la detección de sesgos.

- Riesgo de discriminación: Si el modelo utiliza indirectamente variables que están correlacionadas con atributos protegidos (como origen étnico o género), podría discriminar sin que las variables explícitas estén presentes. Por ejemplo, la variable foreign_worker o incluso el residence_since podrían tener correlaciones con variables protegidas. La falta de transparencia podría ocultar esta discriminación.

### ¿Puede explicarse este modelo a un equipo de riesgo bancario?

Sí, este modelo puede explicarse a un equipo de riesgo bancario, y de hecho, es esencial hacerlo como parte de la explicabilidad de los modelos. Para esto, no basta con mostrar solo la precisión o el recall del modelo; la clave está en el uso de herramientas como SHAP.

- SHAP (SHapley Additive exPlanations): Los gráficos SHAP que generaste son la herramienta perfecta para esta tarea. Permiten transformar la "caja negra" del modelo en un sistema explicable.
  - Identificación de los factores de riesgo: Un analista puede mostrar el gráfico y decir: "La duración del préstamo, el estado de la cuenta corriente y el nivel de ahorros son los factores más importantes que el modelo considera para evaluar el riesgo". Esto alinea las decisiones del modelo con el conocimiento y la intuición del equipo de riesgo.
  - Explicación de casos individuales: SHAP también permite generar explicaciones para una predicción individual. Por ejemplo, si el modelo deniega un préstamo, se puede generar un gráfico para ese cliente específico que muestre qué características (un saldo bajo, una larga duración del préstamo) contribuyeron más a la decisión de alto riesgo.

- Balanceo de errores: La discusión sobre el recall y la precision es también fundamental para el equipo de riesgo. Se puede explicar que el modelo de DNN se eligió por su alto recall para minimizar las pérdidas por impago (Falsos Negativos), lo cual es la prioridad financiera del banco. Se debe ser transparente y admitir que esto conlleva una tasa más alta de rechazos a clientes que sí pagarían (Falsos Positivos), y discutir si ese equilibrio es aceptable.
- Validación de conocimientos: Al presentar los resultados, un equipo de riesgo puede validar si los hallazgos del modelo concuerdan con su experiencia. Por ejemplo, el hecho de que checking_status sea la característica más importante confirma la intuición de que el flujo de caja del cliente es el principal predictor de riesgo.

En conclusión, la combinación de métricas de rendimiento (especialmente recall) y herramientas de explicabilidad como SHAP hace que estos modelos de IA sean no solo útiles, sino también explicables y auditables para un equipo de riesgo bancario. Esto es vital para generar confianza, cumplir con regulaciones y tomar decisiones financieras responsables.

---